In [1]:
import numpy as np
import pandas as pd

import _pickle as cPickle
from copy import deepcopy
from datetime import date, datetime, timedelta
import japanize_matplotlib
import lightgbm as lgb
import matplotlib.pyplot as plt
import pickle
from scipy import stats
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold
import time
from tqdm import tqdm


In [2]:
metadata = pd.read_csv('../input/building_metadata.csv')
weather_train = pd.read_csv('../input/weather_train.csv')

In [3]:
train = pd.read_csv('../input/train.csv')
train = train.merge(metadata,on='building_id',how = 'left')
train = train.merge(weather_train,on=['site_id','timestamp'],how='left')

leaked_df = pd.read_feather('../input/leak.feather')
leaked_df['meter'] = leaked_df['meter'].astype(int)
leaked_df = leaked_df.rename(columns={'meter_reading':'leaked_meter_reading'})

dic = train[['building_id','site_id']]
dic = dic.set_index('building_id').to_dict()
dic = dic['site_id']

leaked_df = leaked_df[leaked_df['leaked_meter_reading'].notnull()]
leaked_df['site_id'] = leaked_df['building_id'].map(dic)

In [4]:
leaked_df = leaked_df.rename(columns={'leaked_meter_reading': 'meter_reading'})

In [5]:
# leaked_df = pd.read_feather('../input/leak_data.feather')#.rename(columns={'meter_reading': 'leaked_meter_reading'})

In [6]:
# 覚え書き
# 連続で同じ値を取るやつを除去
# ただし、同じ値を取るやつが最小値だった場合は除去しない(電気データの場合、最小値=休みの日とかの可能性があるため)

del_list = list()

for building_id in range(1449):
    leaked_df_gb = leaked_df[leaked_df['building_id'] == building_id].groupby("meter")

    for meter, tmp_df in leaked_df_gb:
#         print("building_id: {}, meter: {}".format(building_id, meter))
        data = tmp_df['meter_reading'].values
#         splited_value = np.split(data, np.where((data[1:] != data[:-1]) | (data[1:] == min(data)))[0] + 1)
#         splited_date = np.split(tmp_df.timestamp.values, np.where((data[1:] != data[:-1]) | (data[1:] == min(data)))[0] + 1)
        splited_idx = np.split(tmp_df.index.values, np.where((data[1:] != data[:-1]) | (data[1:] == min(data)))[0] + 1)
        for i, x in enumerate(splited_idx):
            if len(x) > 24:
#                 print("length: {},\t{}-{},\tvalue: {}".format(len(x), x[0], x[-1], splited_value[i][0]))
                del_list.extend(x[1:])
                
                
#         print()

del tmp_df, leaked_df_gb

In [7]:
len(del_list)

134269

In [8]:
# def idx_to_drop(df):
#     drop_cols = []
#     electric_zero = df[(df['meter']==0)&(df['meter_reading']==0)].index.values.tolist()
#     drop_cols.extend(electric_zero)
#     not_summer = df[(df['timestamp'].dt.month!=7)&(df['timestamp'].dt.month!=8)]
#     not_summer['cumsum'] = not_summer.groupby(['building_id','meter'])['meter_reading'].cumsum()
#     not_summer['shifted'] = not_summer.groupby(['building_id','meter'])['cumsum'].shift(48)
#     not_summer['difference'] = not_summer['cumsum']-not_summer['shifted']
#     steam_zero = not_summer[(not_summer['difference']==0) & (not_summer['meter']==2)].index.values.tolist()
#     hotwater_zero = not_summer[(not_summer['difference']==0) & (not_summer['meter']==3)].index.values.tolist()
#     drop_cols.extend(steam_zero)
#     drop_cols.extend(hotwater_zero)
#     del not_summer
#     not_winter = leaked_df[(df['timestamp'].dt.month!=12)&(df['timestamp'].dt.month!=1)]
#     not_winter['cumsum'] = not_winter.groupby(['building_id','meter'])['meter_reading'].cumsum()
#     not_winter['shifted'] = not_winter.groupby(['building_id','meter'])['cumsum'].shift(48)
#     not_winter['difference'] = not_winter['cumsum']-not_winter['shifted']
#     chilled_zero = not_winter[(not_winter['difference']==0) & (not_winter['meter']==1)].index.values.tolist()
#     drop_cols.extend(chilled_zero)
#     return drop_cols

# del_list.extend(idx_to_drop(leaked_df))

In [9]:
del_list_new = leaked_df.loc[del_list].index#query('timestamp < 20160901').index

In [10]:
leaked_df = leaked_df.drop(del_list_new).reset_index(drop=True)

In [11]:
with open('../input/leak_data_drop_bad_rows.pkl', 'wb') as f:
    pickle.dump(leaked_df,f)